In [ ]:
## 자동차 실행을 위한 파일입니다.

import cv2
import time
import numpy as np
import imutils
from pop import Util

# 이미 구한 파라미터
# camera_matrix = np.array([[491.20761393, 0., 251.39564226],[0.,634.00606786, 319.98360268],[  0., 0., 1.]])
# dist_coeffs = np.array([-0.38513139, 0.16790216, -0.02370567,  0.00794167, -0.05845682])

# crop points
x1,y1 = 120,75
x2,y2 = 200, 75
x3, y3 = 260, 230
x4, y4 = 60, 230
hsv_lower = np.array([50, 90, 90])
hsv_upper = np.array([120, 150, 180])




def roi_kim(img):
    vertices = np.array([[x1,y1], [x2,y2], [x3,y3], [x4,y4]],np.int32)
    mask = np.full_like(img,(255,255,255))
    im = cv2.fillPoly(mask, [vertices],(0,0,0))
    roi_image = cv2.bitwise_or(img, im)
    return roi_image
    
    
    
    
def mask_hsv(img, hsv_lower, hsv_upper):
    # hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    # mask = cv.inRange(hsv, hsv_lower, hsv_upper)
    # masked_img = cv.bitwise_and(img, img, mask=mask)
    # cv.imshow("hsv", hsv)

    mask0 = cv2.inRange(img, hsv_lower, hsv_upper)
    # upper mask (170-180)
    # lower_red = np.array([170,50,50])
    # upper_red = np.array([180,200,255])
    # mask1 = cv.inRange(img, lower_red, upper_red)
    # mask= mask0+mask1
    masked_img = cv2.bitwise_and(img, img, mask=mask0)
    return mask0

In [ ]:
# webcam 불러오기
from pop import Pilot

Car = Pilot.AutoCar()
flag = 0
mid = 160


Util.enable_imshow()

cam = Util.gstrmer(width=640,height=480)
camera =cv2.VideoCapture(cam,cv2.CAP_GSTREAMER)

img_counter=0
interval =10
camera_matrix_file = "intrinsic_parameters/camera_matrix.npy"
dist_coeffs_file = "intrinsic_parameters/distortion_coefficients.npy"

camera_matrix=np.load(camera_matrix_file)
dist_coeffs=np.load(dist_coeffs_file)
if not camera.isOpened():
    print("NOT found camera")

count=0

# Car 초기설정 
Car.steering = 0
Car.forward()
Car.setSpeed(99)

#Car.setSpeed(99)

x_avg=0
while True:
    t1= time.time()
    ret,frame = camera.read()
    # Downscale
    scale =50
    wd= int(frame.shape[1]*scale/100)
    ht= int(frame.shape[0]*scale/100)
    dim=(wd,ht)
    frame_ds= cv2.resize(frame,dim,interpolation=cv2.INTER_AREA)

    # Gaussian Blur
    blur = cv2.GaussianBlur(frame_ds, (5,5), 0)


    # Crop
    crop = roi_kim(blur)
    cv2.imshow('roiroi',crop)
    # print(crop.shape)
    
    # Green mask
    img_HSV = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
    lower = np.array([80,90,50])
    upper = np.array([110,200,255])
    mask = cv2.inRange(img_HSV, lower, upper)
    cv2.imshow('mask',mask)
    
    # mask = mask_hsv(img_HSV, hsv_lower, hsv_upper)
    # cv2.imshow('mask',mask)
        
    # gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    # gray2 = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)

        
    # Black & White masking/ blue할 때 150 black으로 100
    # binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV) 
        
    # Drawing Contour
    # binary = cv2.erode(binary,None, iterations=2)
    # binary = cv2.dilate(binary,None, iterations=2)
    cnts= cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts=imutils.grab_contours(cnts)


    # 요서 그 off track일때 코드 생성
    if (len(cnts)!=0):
        c = max(cnts, key=cv2.contourArea)
        cv2.drawContours(frame_ds,[c],-1,(0,255,255),2) ##img, contours, contourIndex, color, thickness

        # 무게중심 찾기 제일 큰 도형의 무게중심
        M = cv2.moments(c)
        if (M['m00']!=0):
            x_moment = int(M['m10']/M['m00'])
            y_moment = int(M['m01']/M['m00'])
            cv2.circle(frame_ds, (x_moment, y_moment), 3, (255,0,0), -1)
        

    # cv2.circle(frame_ds, (x_moment, y_moment), 3, (0,0,255), -1)
    
     
        
    # 결과 출력
    cv2.imshow('vision',frame_ds)
    #cv2.imshow('binary', binary)
    #cv2.imshow('Crop',gray)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
            

    
	# 제어 파트 작성 부분 - PID로 하려다 실패 -> 화면 무게중심 픽셀로 변경
	
    # left
    if  x_moment<(mid - 50) and (flag!=1):
        Car.steering = -0.9
        flag=1
        
    if x_moment < (mid - 30) and x_moment > (mid - 50) and (flag!=3):
        Car.steering = -0.2
        flag = 3

    # right
    if x_moment>(mid + 50) and (flag!=2):
        Car.steering = 0.9
        flag=2
    if x_moment>(mid + 30) and x_moment<(mid + 50) and (flag!=4):
        Car.steering = 0.2
        flag=4

    # center    
    if (x_moment>(mid-30) and x_moment<(mid+30)) and (flag!=0):
        Car.steering = 0
        flag=0
    # count = 0
    
    
camera.release()
cv2.destoryAllWindows()
